In [ ]:
import tensorflow as tf
import collections
import torch
import numpy as np
import pickle

Upload checkpoint.rar, downloaded from https://www.dropbox.com/s/fkoox0t3jwrf92q/checkpoint.rar?dl=0

In [ ]:
# !wget https://www.dropbox.com/s/fkoox0t3jwrf92q/checkpoint.rar?dl=1
!wget https://www.dropbox.com/s/fkoox0t3jwrf92q/checkpoint.rar

--2022-05-07 12:49:51--  https://www.dropbox.com/s/fkoox0t3jwrf92q/checkpoint.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fkoox0t3jwrf92q/checkpoint.rar [following]
--2022-05-07 12:49:51--  https://www.dropbox.com/s/raw/fkoox0t3jwrf92q/checkpoint.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce3de60c0f2c9860f9402b7501e.dl.dropboxusercontent.com/cd/0/inline/Bk0WWiaYkWxREtc8d-ZaHJiLjsp2TWUHOcnGA33jpd-n8Zr2l32cAQEYiqXBHjmiT6KcF1re4a_qTB3haZ0ugNJ_oTTvcOmudXRTj23sfjNDukC9FUhjf7bCLw4kWzFpYwj2ycduAPQ5HaJCpTSiVTihTm6iEfBSImTyaZG1oEqhHA/file# [following]
--2022-05-07 12:49:52--  https://uce3de60c0f2c9860f9402b7501e.dl.dropboxusercontent.com/cd/0/inline/Bk0WWiaYkWxREtc8d-ZaHJiLjsp2TWUHOcnGA33jpd-n8Zr2l32cAQEYiqX

In [ ]:
!unrar x checkpoint.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from checkpoint.rar

Creating    checkpoint                                                OK
Creating    checkpoint/coarse_112                                     OK
Extracting  checkpoint/coarse_112/checkpoint                               0%  OK 
Extracting  checkpoint/coarse_112/coarse.model-1500.data-00000-of-00001       2%  OK 
Extracting  checkpoint/coarse_112/coarse.model-1500.index                  2%  OK 
Extracting  checkpoint/coarse_112/coarse.model-1500.meta.tmp5d5afa1ee1af439dab6e40f5a5a748bc       5%  8% 10% 13% 16% 19% 21% 24% 27% 29% 32% 35% 38% 40% 43% 46% 49% 51% 54% 57% 59% 62% 65% 68% 70% 73% 76% 79% 81% 84% 87% 89% 92% 95% 98% 99%  OK 
All OK


In [ ]:
checkpoint_dir = "checkpoint/coarse_112"

Ref: https://github.com/BIGWangYuDong/UWEnhancement/blob/master/tools/tf2torch.py

In [ ]:
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)

In [ ]:
tfvars = []
init_vars = tf.train.list_variables(ckpt.model_checkpoint_path)

In [ ]:
init_vars

[('main_branch/conv2wb_1/biases', [128]),
 ('main_branch/conv2wb_1/w', [7, 7, 12, 128]),
 ('main_branch/conv2wb_10/biases', [32]),
 ('main_branch/conv2wb_10/w', [5, 5, 32, 32]),
 ('main_branch/conv2wb_100/biases', [32]),
 ('main_branch/conv2wb_100/w', [5, 5, 32, 32]),
 ('main_branch/conv2wb_1000/biases', [32]),
 ('main_branch/conv2wb_1000/w', [5, 5, 32, 32]),
 ('main_branch/conv2wb_11/biases', [3]),
 ('main_branch/conv2wb_11/w', [3, 3, 32, 3]),
 ('main_branch/conv2wb_111/biases', [3]),
 ('main_branch/conv2wb_111/w', [3, 3, 32, 3]),
 ('main_branch/conv2wb_1111/biases', [3]),
 ('main_branch/conv2wb_1111/w', [3, 3, 32, 3]),
 ('main_branch/conv2wb_2/biases', [128]),
 ('main_branch/conv2wb_2/w', [5, 5, 128, 128]),
 ('main_branch/conv2wb_3/biases', [128]),
 ('main_branch/conv2wb_3/w', [3, 3, 128, 128]),
 ('main_branch/conv2wb_4/biases', [64]),
 ('main_branch/conv2wb_4/w', [1, 1, 128, 64]),
 ('main_branch/conv2wb_5/biases', [64]),
 ('main_branch/conv2wb_5/w', [7, 7, 64, 64]),
 ('main_branch/c

Naming scheme is `main_branch/conv layer/weight or bias`.

For reference, `state_dict` of my Pytorch implementation is:

```
cmg.conv1.weight torch.Size([128, 12, 7, 7])
cmg.conv1.bias torch.Size([128])
cmg.conv2.weight torch.Size([128, 128, 5, 5])
cmg.conv2.bias torch.Size([128])
cmg.conv3.weight torch.Size([128, 128, 3, 3])
cmg.conv3.bias torch.Size([128])
cmg.conv4.weight torch.Size([64, 128, 1, 1])
cmg.conv4.bias torch.Size([64])
cmg.conv5.weight torch.Size([64, 64, 7, 7])
cmg.conv5.bias torch.Size([64])
cmg.conv6.weight torch.Size([64, 64, 5, 5])
cmg.conv6.bias torch.Size([64])
cmg.conv7.weight torch.Size([64, 64, 3, 3])
cmg.conv7.bias torch.Size([64])
cmg.conv8.weight torch.Size([3, 64, 3, 3])
cmg.conv8.bias torch.Size([3])
wb_refiner.conv1.weight torch.Size([32, 6, 7, 7])
wb_refiner.conv1.bias torch.Size([32])
wb_refiner.conv2.weight torch.Size([32, 32, 5, 5])
wb_refiner.conv2.bias torch.Size([32])
wb_refiner.conv3.weight torch.Size([3, 32, 3, 3])
wb_refiner.conv3.bias torch.Size([3])
ce_refiner.conv1.weight torch.Size([32, 6, 7, 7])
ce_refiner.conv1.bias torch.Size([32])
ce_refiner.conv2.weight torch.Size([32, 32, 5, 5])
ce_refiner.conv2.bias torch.Size([32])
ce_refiner.conv3.weight torch.Size([3, 32, 3, 3])
ce_refiner.conv3.bias torch.Size([3])
gc_refiner.conv1.weight torch.Size([32, 6, 7, 7])
gc_refiner.conv1.bias torch.Size([32])
gc_refiner.conv2.weight torch.Size([32, 32, 5, 5])
gc_refiner.conv2.bias torch.Size([32])
gc_refiner.conv3.weight torch.Size([3, 32, 3, 3])
gc_refiner.conv3.bias torch.Size([3])
```

By observation, biases are all vectors and do not require additional transforms. Weights are pretty much in inverted order.

Can format into a correct state dict and load it.


In [ ]:
layer_mapper = {
    # Confidence map generator
    "conv2wb_1": "cmg.conv1",
    "conv2wb_2": "cmg.conv2",
    "conv2wb_3": "cmg.conv3",
    "conv2wb_4": "cmg.conv4",
    "conv2wb_5": "cmg.conv5",
    "conv2wb_6": "cmg.conv6",
    "conv2wb_7": "cmg.conv7",
    "conv2wb_77": "cmg.conv8",
    # WB refiner
    "conv2wb_9": "wb_refiner.conv1",
    "conv2wb_10": "wb_refiner.conv2",
    "conv2wb_11": "wb_refiner.conv3",
    # CE refiner
    "conv2wb_99": "ce_refiner.conv1",
    "conv2wb_100": "ce_refiner.conv2",
    "conv2wb_111": "ce_refiner.conv3",
    # GC refiner
    "conv2wb_999": "gc_refiner.conv1",
    "conv2wb_1000": "gc_refiner.conv2",
    "conv2wb_1111": "gc_refiner.conv3",
}

In [ ]:
wt_example = tf.train.load_variable(ckpt.model_checkpoint_path, "main_branch/conv2wb_999/w")

In [ ]:
type(wt_example)

numpy.ndarray

In [ ]:
wt_example.shape

(7, 7, 6, 32)

In [ ]:
wt_example.transpose().shape

(32, 6, 7, 7)

## Reshape and assign to OrderedDict

In [ ]:
wnb = collections.OrderedDict()

In [ ]:
for name, shape in init_vars:
    _, og_layer_name, layer_type = name.split("/")
    new_layer_name = layer_mapper.get(og_layer_name)

    param = tf.train.load_variable(ckpt.model_checkpoint_path, name)
    param = np.ascontiguousarray(np.transpose(param))
    param = torch.from_numpy(param)

    if layer_type == "biases":
        new_layer_name = new_layer_name + ".bias"
    elif layer_type == "w":
        new_layer_name = new_layer_name + ".weight"
    else:
        pass

    wnb[new_layer_name] = param

    print(f"{name} saved as {new_layer_name}")

main_branch/conv2wb_1/biases saved as cmg.conv1.bias
main_branch/conv2wb_1/w saved as cmg.conv1.weight
main_branch/conv2wb_10/biases saved as wb_refiner.conv2.bias
main_branch/conv2wb_10/w saved as wb_refiner.conv2.weight
main_branch/conv2wb_100/biases saved as ce_refiner.conv2.bias
main_branch/conv2wb_100/w saved as ce_refiner.conv2.weight
main_branch/conv2wb_1000/biases saved as gc_refiner.conv2.bias
main_branch/conv2wb_1000/w saved as gc_refiner.conv2.weight
main_branch/conv2wb_11/biases saved as wb_refiner.conv3.bias
main_branch/conv2wb_11/w saved as wb_refiner.conv3.weight
main_branch/conv2wb_111/biases saved as ce_refiner.conv3.bias
main_branch/conv2wb_111/w saved as ce_refiner.conv3.weight
main_branch/conv2wb_1111/biases saved as gc_refiner.conv3.bias
main_branch/conv2wb_1111/w saved as gc_refiner.conv3.weight
main_branch/conv2wb_2/biases saved as cmg.conv2.bias
main_branch/conv2wb_2/w saved as cmg.conv2.weight
main_branch/conv2wb_3/biases saved as cmg.conv3.bias
main_branch/con

In [ ]:
with open("waternet-exported-state-dict.pkl", "wb") as f:
    pickle.dump(wnb, f)